# 전환이 일어난 유저(cart or purchase)에게 본 수(view)로 추천


# 라이브러리 로드

In [1]:
!pip install implicit
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 2.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 2.1 MB/s 
     |████████████████████████████████| 1.6 MB 63.7 MB/s 


In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit

from tqdm import tqdm

# 데이터 로드

In [3]:
try:
  path = 'C:/Users/User/Desktop/AIB_13/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')
except:
  path = '/content/drive/MyDrive/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')

In [4]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,-251657396,NaN,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,-780140327,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,-1904213353,furniture.living_room.sofa,NaN,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,1518338663,computers.notebook,lenovo,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,-1769995873,electronics.smartphone,apple,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [5]:
data = df[['user_id','product_id','event_type']]

# 데이터 전처리

## cart, purchase가 존재하는 유저 데이터 추출

1. event_type 컬럼에 cart가 존재하는 user_id 추출
2. event_type 컬럼에 purchase가 존재하는 user_id 추출
3. 위에서 구한 id 합치기

In [6]:
cart_user_id = set(data.loc[data['event_type'] == 'cart','user_id'].unique())
purchase_user_id = set(data.loc[data['event_type'] == 'purchase', 'user_id'].unique())

all_id = cart_user_id.union(purchase_user_id)

In [7]:
data = data.loc[data['user_id'].isin(all_id)].reset_index(drop=True)

In [8]:
data['event_type'] = data['event_type'].astype('object')

## view 적용

In [9]:
grouped = data.groupby(['user_id','product_id','event_type'])['event_type'].count()
grouped = pd.DataFrame(grouped).rename(columns={'event_type' : 'count'}).reset_index()

In [10]:
table = grouped.pivot_table(index=['user_id','product_id'], columns=['event_type'], values=['count'])
table = table.reset_index()
table.columns = ['user_id','product_id','cart','purchase','view']
table = table.fillna(0)

In [11]:
table['cart'] = table['cart'].astype('int')
table['purchase'] = table['purchase'].astype('int')
table['view'] = table['view'].astype('int')

In [12]:
table.head()

,user_id,product_id,cart,purchase,view
0,264649825,8500081,0,0,1
1,264649825,8500083,0,1,12
2,264649825,8500084,0,1,4
3,264649825,8500086,0,0,2
4,264649825,27700136,0,0,1


table을 확인했을때 view 값이 0인 값이 존재했습니다.  
이것의 의미는 수집된 시점이전에 view를 했거나,  
제품의 상세 페이지에 방문하지 않고 바로 전환이 일어났다고 가정하여, 평균값이 `2`를 넣어주겠습니다.

In [13]:
table.loc[table['view'] == 0, 'view'] = 2

In [14]:
data = table[['user_id','product_id','view']]

In [15]:
data.head()

,user_id,product_id,view
0,264649825,8500081,1
1,264649825,8500083,12
2,264649825,8500084,4
3,264649825,8500086,2
4,264649825,27700136,1


# Item Lookup(product lookup) 테이블 만들기

In [16]:
product_lookup = df[['product_id','category_code','brand']].drop_duplicates('product_id').reset_index(drop=True).sort_values('product_id')
product_lookup.head()

,product_id,category_code,brand
151915,1000978,electronics.smartphone,NaN
8437,1001588,electronics.smartphone,meizu
85152,1001606,electronics.smartphone,apple
32556,1002042,electronics.smartphone,samsung
9400,1002062,electronics.smartphone,samsung


# Rating Matrix 만들기

In [17]:
num_user = data['user_id'].nunique()
num_item = data['product_id'].nunique()

num_user, num_item

(481458, 148488)

In [18]:
users = list(np.sort(data['user_id'].unique()))
products = list(data['product_id'].unique())
view = list(data['view'])

rows = data['user_id'].astype('category').cat.codes
data['user_id_code'] = data['user_id'].astype('category').cat.codes

cols = data['product_id'].astype('category').cat.codes
data['product_id_code'] = data['product_id'].astype('category').cat.codes

len(users), len(products), len(view)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(481458, 148488, 7942703)

In [19]:
user_item_matrix = sparse.csr_matrix((view, (rows, cols)), shape=(num_user, num_item))
user_item_matrix

<481458x148488 sparse matrix of type '<class 'numpy.int64'>'
	with 7942703 stored elements in Compressed Sparse Row format>

# 추천 시스템 class 구현

In [20]:
from implicit.als import AlternatingLeastSquares
from sklearn import metrics
class MyALS():
  """
  implicit 라이브러리를 이용하여 필요한 기능을 담았습니다.

  Parameters
  ----------
  model : implicit 라이브러리의 AlternatingLeastSquares() 클래스로 생성된 인스턴스
  user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
  item_lookup : item(product)에 대한 정보를 담은 테이블
  data : User-Item Matrix를 만든 원본데이터

  """
  def __init__(self, model, user_item_matrix, item_lookup, data):

    self.model = model
    self.metrics_model = model
    self.user_item_matrix = user_item_matrix
    self.item_lookup = item_lookup
    self.data = data

  def fit(self, user_item_matrix):
    """
    행렬분해의 ALS를 이용하여 모델을 학습합니다.

    Parameters
    ----------
    user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
    """
    self.model.fit(user_item_matrix)
    
  def _user_id_2_code(self, user_id):
    """
    입력받은 user_id를 User_Item_Matrix에 있는 user_id_code로 바꾸어주는 함수

    Parameters
    ----------
    user_id : 유저 ID

    """
    user_id_code = self.data.loc[self.data['user_id'] == user_id, 'user_id_code'].unique()[0]
    return user_id_code

  def _product_id_2_code(self, product_id):
    """
    입력받은 product_id를 User-Item-Matrix에 있는 product_id_code로 바꾸어주는 함수

    Parameters
    ----------
    product_id : 상품 ID
    """
    product_id_code = self.data.loc[self.data['product_id'] == product_id, 'product_id_code'].unique()[0]
    return product_id_code

  def _code_2_product_id(self, product_id_code):
    """
    입력받은 product_id_code를 User-Item-Matrix에 있는 product_id로 바꾸어주는 함수

    Parameters
    ----------
    product_id_code : 상품 ID code    
    """
    product_id = self.data.loc[self.data['product_id_code'] == product_id_code, 'product_id'].unique()[0]
    return product_id

  def get_recom_product(self, user_id, n = 10):
    """  
    user_id에 맞는 product를 n개 만큼 추천하여 데이터프레임 형태로 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    n : 추천 받게 될 item의 수
    """ 

    # user_id_2_code 함수를 이용하여 유저의 ID를 user_id_code로 변환합니다
    user_id_code = self._user_id_2_code(user_id)
  
    # model의 recommend를 이용하여 추천받는 제품의 id를 추출합니다.
    # 이때 추천 받는 제품의 id는 product_id가 아니라 product_id_code 입니다.
    recommended = self.model.recommend(user_id_code, self.user_item_matrix[user_id_code], N=n)[0]
    #결과를 담을 리스트를 초기화 합니다.
    results = []
    # 추천 받은 id를 돌면서 item_lookup 테이블에서 해당 product의 정보를 찾아 결과에 담습니다.
    for product_id_code in recommended:
      
      recommended_product_id = self._code_2_product_id(product_id_code)
      result = self.item_lookup.loc[self.item_lookup['product_id'] == recommended_product_id]
      results.append(result)
      
    return pd.concat(results)
  
  def get_user_topN_product(self,user_id,column, n = 20):
    """
    유저가 특정 기준값이 높은 제품 N개를 반환

    Parameters
    ----------
    user_id : 유저 ID
    column : 값을 확인할 기준이 되는 컬럼
    n : 반환할 Item 수
    """
    #입력받은 user_id 를 기준으로 column이 높은 순으로 정렬하여 product_id를 추출
    product_ids = self.data.loc[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n]['product_id'].values
    #입력받은 user_id 를 기준으로 column이 높은 순으로 정렬하여 column을 추출
    product_values = self.data[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n][column].values

    results = []
    #item_lookup 테이블에서 id에 맞는 데이터프레임을 찾음
    for i in product_ids:
      result = self.item_lookup.loc[self.item_lookup['product_id'] == i]
      results.append(result)

    #결과를 확인하기 쉽게 데이터프레임으로 반환
    frame = pd.concat(results)
    frame[column] = product_values

    return frame

  def get_explain(self, user_id, item_id, column):
    """
    사용자에게 제품이 추천된 이유를 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    item_id : Item(product) ID
    column : 확인할 컬럼
    """
    #입력받은 user_id, item_id를 user_id_code, product_id_code로 바꾸어줌
    user_id_code = self._user_id_2_code(user_id)
    product_id_code = self._product_id_2_code(item_id)

    #implicit라이브러리의 explain 함수를 사용하여 결과값을 반환
    total_score, top_contributions, user_weights = self.model.to_cpu().explain(user_id_code, self.user_item_matrix, product_id_code)

    results = []
    categorys = []
    brands = []
    scores = []
    # id에 해당하는 user_id, product_id, column, category, brand를 찾기
    for id_, score_ in top_contributions:
      product_id = self._code_2_product_id(id_)
      result = self.data.loc[(self.data['product_id'] == product_id) & (self.data['user_id'] == user_id)][['user_id','product_id',column]]
      category = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'category_code'].unique()[0]
      brand = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'brand'].unique()[0]

      results.append(result)
      categorys.append(category)
      brands.append(brand)
      scores.append(score_)

    #결과를 확인하기 쉽게 데이터프레임으로 반환
    frame = pd.concat(results)
    frame['score'] = scores
    frame['category'] = categorys
    frame['brand'] = brands
    
    frame = frame[['user_id', 'product_id','category','brand',column, 'score']]
    return frame, total_score

  def _get_train_test(self,percentage=.2, seed=42):
    """
    score를 구하기 위하여 train, test 데이터를 나누어주는 함수
    파라미터로 들어오는 percentage만큼 train_set의 값을 0으로 만들어 줌
    test_set는 기존의 User-Item Matrix에서 0이 아닌값으로 모두 1로 만들어 줌

    Parameters
    ----------
    percentage : 감추고 싶은 데이터의 비율
    seed : random seed

    """
    #원본 데이터를 test_set, train_set에 복사
    test_set = self.user_item_matrix.copy()
    train_set = self.user_item_matrix.copy()

    #relevant(선호 혹은 평가)여부를 확인하기 위하여 test_set에서 0이 아닌값을 1로 만들어 줌 
    test_set[test_set != 0] = 1

    #train_set에서 0이 아닌 x축, y축을 추출
    nonzero_idxs = train_set.nonzero()
    #x, y를 짝을지어 저장
    nonzero_pairs = list(zip(nonzero_idxs[0], nonzero_idxs[1]))

    #랜덤 시드를 적용
    random.seed(seed)
    #주어진 비율로 샘플을 추출
    n_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, n_samples)

    
    user_idxs = [index[0] for index in samples]
    item_idxs = [index[1] for index in samples]

    #샘플에 해당 하는 값들을 평가한적이 없도록 보이기 위하여 0으로 감춤
    train_set[user_idxs, item_idxs] = 0
    
    train_set.eliminate_zeros()

    self.zero_user_idxs = user_idxs
    self.zero_item_idxs = item_idxs
    self.train_set = train_set
    self.test_set = test_set
    

  def get_score(self, percentage =.2, seed=42, k = 10, method='hit_at_k', n_samples = 10000):
    """
    train_set로 학습하고 test_set와 비교하여 method 파라미터를 이용하여 추천시스템의 성능을 평가하는 함수
    
    Parameters
    ----------
    percentage : 감추고 싶은 데이터의 비율, default = .2
    seed : random seed, default = 42
    k : 추천할 아이템의 수, default = 10
    method : 평가 지표 - hit_at_k, precision_at_k
    n_samples : 평가할 user의 수, default = 10000

    """
    # 입력받은 n_samples가 최대값이상 이면 최대값으로 적용
    max_num = self.data['user_id_code'].nunique()
    if max_num <= n_samples:
      n_samples = max_num

    # train, test로 데이터가 분리
    self._get_train_test(percentage, seed)
    # metrics_model이 학습
    self.metrics_model.fit(self.train_set)
    # 샘플 user를 랜덤으로 추출
    random_state = np.random.RandomState(seed)
    user_id_code_samples = random_state.choice(self.data['user_id_code'], n_samples)

    #method 방법에 따라 scores 값을 반환
    if method == 'hit_at_k':
      scores = self._hit_at_k(user_id_code_samples, k)
    elif method =='presicion_at_k':
      scores = self._precision_at_k(user_id_code_samples, k)
    return scores

  def _hit_at_k(self, user_id_code_samples, k):
    """
    k개의 추천 중 relevant한것(존재)이 있다면 1, 아니면 0을 반환하여 측정
    추천을 받은 user 수 만큼 나누어 평균을 반환

    Parameters
    ----------
    user_id_code_samples : 샘플링한 유저의 ID 리스트
    k : 추천할 아이템의 수
    """
    
    scores = []
    #입력받은 user_id_code_samples를 돌면서
    for user_id_code_sample in tqdm(user_id_code_samples):
      #해당 유저에게 추천하는 아이템을 추출
      recommedation_ids = self.metrics_model.recommend(user_id_code_sample, self.train_set[user_id_code_sample], N=k)[0]

      results = []
      # 추천 받은 아이템이 유저가 선호(혹은 평가)했는지 확인
      for id_ in recommedation_ids:
        result = self.test_set[user_id_code_sample, id_]
        results.append(result)
      # 만약 결과 리스트안에 1이 있다면 1을 입력, 아니면 0을 입력
      if 1 in results:
        scores.append(1)
      else:
        scores.append(0)
        #결과를 평균내어 반환
    return np.mean(scores)

  def _precision_at_k(self, user_id_code_samples, k):
    """
    k개의 추천 중 사용자가 relevant(선호 혹으 평가)한 아이템이 얼마나 존재하는지 측정
    추천을 받은 user 수 만큼 나누어 평균을 반환

    Parameters
    ----------
    user_id_code_samples : 샘플링한 유저의 ID 리스트
    k : 추천할 아이템의 수
    """
    scores = []
    #입력받은 user_id_code_samples를 돌면서
    for user_id_code_sample in tqdm(user_id_code_samples):
      #해당 유저에게 추천하는 아이템을 추출
      recommedation_ids = self.metrics_model.recommend(user_id_code_sample, self.train_set[user_id_code_sample], N=k)[0]
      results = []
       # 추천 받은 아이템이 유저가 선호(혹은 평가)했는지 확인
      for id_ in recommedation_ids:
        result = self.test_set[user_id_code_sample, id_]
        results.append(result)
      # 유저가 추천받은 아이템들을 얼마나 선호(혹은 평가)했는지 추출
      scores.append(np.mean(results))
      # 결과를 평균내어 반환
    return np.mean(scores)

# 추천하기

In [21]:
model = MyALS(AlternatingLeastSquares(), user_item_matrix, product_lookup, data)

In [22]:
model.fit(user_item_matrix)

  0%|          | 0/15 [00:00<?, ?it/s]

In [23]:
data.sort_values('view', ascending=False)

,user_id,product_id,view,user_id_code,product_id_code
3944605,523974502,5100563,311,186199,17608
1038402,513340366,1003306,276,37864,132
2240329,515541823,5100789,274,92052,17770
14481,512365995,1004767,207,908,796
6751136,555218597,1201513,198,361429,1457
...,...,...,...,...,...
3329218,519224481,3200173,1,149925,9733
3329217,519224481,3100955,1,149925,9635
3329216,519224481,3100906,1,149925,9601
3329215,519224481,1005207,1,149925,1160


In [24]:
table.loc[table['user_id'] == 523974502]

,user_id,product_id,cart,purchase,view
3944579,523974502,1004210,0,0,2
3944580,523974502,1004249,1,5,12
3944581,523974502,1004250,0,1,4
3944582,523974502,1004259,1,1,2
3944583,523974502,1004777,0,0,4
3944584,523974502,1004834,0,0,1
3944585,523974502,1004856,0,0,1
3944586,523974502,1005079,0,1,2
3944587,523974502,1005081,0,1,2
3944588,523974502,1005104,0,1,3


In [25]:
user = 523974502
column = 'view'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,view
1724,5100563,electronics.clocks,samsung,311
635,5100718,electronics.clocks,samsung,24
388,5100737,electronics.clocks,samsung,13
113,1004249,electronics.smartphone,apple,12
51555,5100862,electronics.clocks,NaN,9
88,1005115,electronics.smartphone,apple,7
1878,5100738,electronics.clocks,samsung,6
556,1004250,electronics.smartphone,apple,4
1104,1004777,electronics.smartphone,xiaomi,4
122,5100816,NaN,xiaomi,3


In [26]:
model.get_recom_product(user)

,product_id,category_code,brand
51950,5100861,electronics.clocks,NaN
884,5100551,electronics.clocks,xiaomi
2740,5100721,electronics.clocks,huawei
8033,5100722,electronics.clocks,huawei
53090,5100863,electronics.clocks,NaN
213,1004209,electronics.smartphone,samsung
579,5100376,electronics.clocks,xiaomi
668,1005124,electronics.smartphone,apple
1146,5100846,electronics.clocks,xiaomi
124455,5100874,electronics.clocks,NaN


In [27]:
explain_frame, total_score = model.get_explain(user, 1002532, column)
explain_frame

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:29: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


,user_id,product_id,category,brand,view,score
3944605,523974502,5100563,electronics.clocks,samsung,311,0.041558
3944608,523974502,5100718,electronics.clocks,samsung,24,0.029982
3944582,523974502,1004259,electronics.smartphone,apple,2,0.024121
3944597,523974502,3300349,NaN,redmond,2,0.019911
3944593,523974502,1005144,electronics.smartphone,apple,3,0.012463
3944614,523974502,5100862,electronics.clocks,NaN,9,0.007218
3944583,523974502,1004777,electronics.smartphone,xiaomi,4,0.004489
3944590,523974502,1005115,electronics.smartphone,apple,7,0.003786
3944589,523974502,1005105,electronics.smartphone,apple,3,0.003760
3944595,523974502,1305215,computers.notebook,apple,2,0.003511


# 성능 평가하기

In [28]:
scores = model.get_score(method='hit_at_k')
print()
print('hit@k(10) : ',round(scores, 2))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:11<00:00, 845.61it/s]


hit@k(10) :  0.47


In [29]:
scores = model.get_score(method='presicion_at_k')
print()
print('precision@k(10) : ',round(scores, 2))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:12<00:00, 792.44it/s]


precision@k(10) :  0.08
